<center> <img src="https://www.austral.edu.ar/ingenieria-posgrados/wp-content/uploads/2018/06/ua-ingenieria-color-logo.png"/> </center>

## Text Mining
## Proyecto: Clasificador de TED Talks





### 01.Preprocesamiento

#### 01.00 - Preparación del Entorno

In [1]:
#Instalación de Librerias y recursos
! pip install -U spacy
! pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.0.0/en_core_web_lg-3.0.0.tar.gz
!pip install unidecode
#Importamos Librerias:

#Generales:
import os
import os.path
import pprint
import string
import unidecode
import re
import bs4
import time
import requests
import json
import random
import warnings
import gc
import pandas as pd
import numpy as np
import plotly.express as px
import ast
import matplotlib.pyplot as plt
import seaborn as sns

#WebScraping
from multiprocessing import Manager, Process
from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import drive

#Particulares
import nltk
from nltk.text import Text
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.collocations import BigramCollocationFinder,BigramAssocMeasures
import spacy
from spacy import displacy

#Modelo
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics.pairwise import cosine_similarity

#No Warnings
warnings.filterwarnings('ignore')

nltk.download('all')
pp = pprint.PrettyPrinter(indent=4, compact=True)
nlp = spacy.load("en_core_web_lg")

#Variables
ted_file = 'Ted_Talk.csv'

language = 'english'

language_stops = set(stopwords.words(language))
language_stops.update(string.punctuation)

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.0.6)
     |████████████████████████████████| 778.7MB 17kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-3.0.0-cp37-none-any.whl size=778750482 sha256=c9d42fc116e4be952b6b98c46bec03aaf1d6f95fda51d1aa4b628580f5535d6e
  Stored in directory: /root/.cache/pip/wheels/76/d4/24/e2bbc81bef966db10207f8280a6cdfd45d7a1d2f97657ab1a5
Successfully built en-core-web-lg
     |████████████████████████████████| 245kB 4.0MB/s 
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading

#### 01.01 - Funciones Principales

In [2]:
def remove_stop_words(text):
  """
    Remueve stop words en inglés

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar

    Returns
    -------
    list
      lista de palabras sin los stop words
  """
  return [token for token in text if token.lower() not in language_stops]

In [3]:
def lematize_words(text):
  """
    Lematización de palabras - aplica lematización de palabras sobre un set de tokens

    Attributes
    ----------
    text: list
      lista de palabras (tokens) sobre los cuales se aplicará la lematización
    
    Returns
    -------
    list
      lista con todas las lematizaciones de las palabras
  """
  doc = nlp(" ".join(text))
  return [token.lemma_ for token in doc]

In [4]:
def remove_meaningless_words(text):
  """
    Remueve palabras sin significado

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar
    
    Returns
    -------
    list
      lista de palabras filtrada en base a expresiones regulares
  """
  patterns = [r"(^={1,}=$)", r'\u200b']
  tokens = text
  for pattern in patterns:
    regexp = re.compile(pattern)
    tokens = [token for token in tokens if not regexp.search(token)]
  return tokens

In [5]:
def clean_short_words(text):
  """
    Limpia palabras con longitud 1

    Attributes
    ----------
    text: str
      documento a tokenizar
    
    Returns
    -------
    list
      lista de tokens
  """
  return [word for word in text if len(word) > 1]

In [6]:
def tokenize(text, mode='word'): 
  """
    Tokenización de documento - tokeniza un documento por palabra o por oración

    Attributes
    ----------
    text: str
      documento a tokenizar
    mode: str, optional
      método de tokenización (default: 'word' (por palabra))
    
    Returns
    -------
    list
      lista de tokens 
    
    Raises
    ------
      Exception
        si el mode no es 'word' o 'sentence'
  """
  if mode == 'word':
    return word_tokenize(text, language=language)
  elif mode == 'sentence':
    return sent_tokenize(text, language=language)
  else:
    raise Exception('metodo de tokenizacion no encontrado')

In [7]:
def similarity_btw_docs(matrix):
  """
    Similitud entre documentos - calcula la similitud entre documentos utilizando Similitud del Coseno

    Attributes
    ----------
    matrix: scipy matrix
      Matriz dispersa para calcular la similaridad

    Returns
    -------
    pd.DataFrame
      retorna un dataframe con el grado de similaridad entre documentos (de 0 a 1)
  """
  matrix_simil = cosine_similarity(matrix)
  return pd.DataFrame(matrix_simil)

In [8]:
def pre_procesamiento_texto(text):
  """
    Pre-procesamiento y obtención de las 20 palabras más significativas

    Attributes
    ----------
    text: str
      documento a analizar

    Returns
    -------
    pd.DataFrame
      retorna un dataframe con las 20 palabras que más se repiten y su frecuencia
  """
  tokenized = tokenize(text)
  without_stops = remove_stop_words(tokenized)
  meaningfull_tokens = remove_meaningless_words(without_stops)
  without_short_words = clean_short_words(meaningfull_tokens)
  lematized_words = lematize_words(without_short_words)
  return lematized_words

In [9]:
def remove_character(serie, char):
  """
    Pre-procesamiento y obtención de las 20 palabras más significativas

    Attributes
    ----------
    serie: pd.Serie
      columna de dataframe a modificar
    char: char
      caracter a remover

    Returns
    -------
    pd.Serie
      retorna una serie
  """
  return serie.str.replace(char, '')

In [10]:
def get_urls(page_list):
  """
    Obtencion de las urls de las charlas TED

    Attributes
    ----------
    page_list: list
      lista de paginas de charlas TED

    Returns
    -------
    list
      retorna una lista con todas las urls de las charlas TED
  """
  urls = ["https://www.ted.com" + url.select("div.media__image a.ga-link")[0].get("href") for url in page_list]
  return urls

In [11]:
def get_transcript(url, count):
  """
    Obtiene la transcripcion de una determinada charla ted

    Attributes
    ----------
    url: str
      url de la charla TED
    count: int
      indice de la pagina url

    Returns
    -------
    str
      retorna una cadena de caracteres con la transcripcion de la charla TED
  """
  transcript = ""
  transcript_res = requests.get(url, headers = {'User-agent': 'your bot 0.1'})
  soup = BeautifulSoup(transcript_res.text)
  e = soup.select('div.Grid.Grid--with-gutter.p-b:4')

  for  e_  in e:
    classes = e_.get('class')
    text = e_.select('p')[0].text
    transcript += text.strip().replace('\t', '').replace('\n', ' ')
                                
  if (transcript_res.status_code!=200) or (transcript_res.text=='') or (transcript==''):
    count_=0
    while  count_ < 3: 
      time.sleep(random.randint(0,900)/1000)
      transcript_res = requests.get(url, headers = {'User-agent': 'your bot 0.1'})
      soup = BeautifulSoup(transcript_res.text)
      e = soup.select('div.Grid.Grid--with-gutter.p-b:4')

      for  e_  in e:
        classes = e_.get('class')
        text = e_.select('p')[0].text
        transcript += text.strip().replace('\t', '').replace('\n', ' ')

      count_ += 1
      if (transcript_res.status_code==200) and (transcript_res.text!='') and (transcript!=''):
        break

  return transcript

In [12]:
def get__json_obj(url):
  """
    Obtiene el objecto JSON de una respectiva URL

    Attributes
    ----------
    url: str
      url a analizar

    Returns
    -------
    str
      retorna una cadena de caracteres que representa el objeto JSON de la URL
  """
  res = requests.get(url.strip(), headers = {'User-agent': 'your bot 0.1'})
  start_index = res.text.find('<script data-spec="q">q("talkPage.init",')
  end_index = res.text[start_index:].find(')</script>')
  script_tag = res.text[start_index: start_index + end_index]
  return script_tag[len('<script data-spec="q">q("talkPage.init",'):]

In [13]:
def get_value(l, m):
  """
    Obtiene el valor de un elemento HTML

    Attributes
    ----------
    l: list
      elementos
    m: s
      metadata

    Returns
    -------
    str
      retorna el valor del elemento HTML
  """
  for i in l:
    try:
      m = m[i]
    except: 
      return ''
  return m

In [14]:
def html_to_text(html):
  """
    Convierte un valor HTML a cadena de caracteres

    Attributes
    ----------
    html: str
      valor de elemento HTML

    Returns
    -------
    str
      retorna la representacion del valor del elemento HTML en cadena de caracdteres
  """
  if str(html)!='nan':
    soup = BeautifulSoup(html)
    return soup.get_text()
  else: 
    return html

In [15]:
def get_elements_dict_from_url(count, url, json_obj):
  """
    Generacion de diccionario de elementos de una URL

    Attributes
    ----------
    count: int
      indice de la URL dentro del listado de URLs
    url: str
      url a analizar
    json_obj: str
      objeto JSON de la URL

    Returns
    -------
    dict
      retorna un diccionario que contiene todos los elementos HTML con sus respectivos valores
  """
  metadata = json.loads(json_obj)["__INITIAL_DATA__"]
  language = get_value(["language"], metadata)
  url__transcript = url + "/transcript?language=" + language
  temp = get_value(["talks", 0, "recorded_at"], metadata)
  t = get_value(["talks", 0, "player_talks", 0, "published"], metadata)

  d = dict()
  d["language"]  =  language
  d["talk__id"]  =  get_value(["current_talk"], metadata)
  d["talk__name"]  =  get_value(["talks", 0, "title"], metadata)
  d["talk__description"]  =  get_value(["description"], metadata)
  d["view_count"]  =  get_value(["viewed_count"], metadata)
  d["duration"]  =  get_value(["talks", 0, "duration"], metadata)
  d["transcript"]  =  get_transcript(url__transcript,count)
  d["video_type_name"]  =  get_value(["talks", 0, "video_type", "name"], metadata)
  d["event"]  =  get_value(["event"], metadata)               
  d["speaker__id"]  =  get_value(["speakers", 0, "id"], metadata)                        
  d["speaker__name"]  =  get_value(["talks", 0, "speaker_name"], metadata)
  d["speaker__description"]  =  get_value(["speakers", 0, "description"], metadata)
  d["speaker__who_he_is"]  =  get_value(["speakers", 0, "whotheyare"], metadata)
  d["speaker__why_listen"]  =  html_to_text(get_value(["speakers", 0, "whylisten"], metadata))
  d["all_speakers_details"]  =  get_value(["speakers"], metadata)                       
  d["recording_date"]  =  temp  if temp==None  else temp[:10]                        
  d["published_timestamp"]  =  datetime.utcfromtimestamp(int(t)).strftime('%Y-%m-%d %H:%M:%S')                      
  d["talks__tags"]  =  get_value(["talks", 0, "tags"], metadata)
  d["number_of__tags"]  =  len(get_value(["talks", 0, "tags"], metadata) or "")                       
  d["native_language"]  =  get_value(["talks", 0, "player_talks", 0, "nativeLanguage"], metadata)                   
  d["url__webpage"]  =  get_value(["url"], metadata)                    
  d["talk__more_resources"]  =  get_value(["talks", 0, "more_resources"], metadata)
  d["number_of__talk__more_resources"]  =  len(get_value(["talks", 0, "more_resources"], metadata) or "")
  d["talk__recommendations__blurb"]  =  get_value(["talks", 0, "recommendations", "blurb"], metadata)                    
  d["talk__recommendations"]  =  get_value(["talks", 0, "recommendations", "rec_lists"], metadata)
  d["number_of__talk__recommendations"]  =  len(get_value(["talks", 0, "recommendations", "rec_lists"], metadata) or "")
  d["related_talks"]  =  get_value(["talks", 0, "related_talks"], metadata)
  d["number_of__related_talks"]  =  len(get_value(["talks", 0, "related_talks"], metadata) or "")

  return d

In [16]:
def download(urls, id_, csv_list):
  """
    Descarga toda la informacion respecto a las charlas TED

    Attributes
    ----------
    urls: list
      lista de URLs a descargar
    id_: int
      lorem ipsum dolor
    csv_list: list
      csv donde se guardara la informacion respecto a las charlas ted
  """
  for count, url in enumerate(urls):
    json_obj = get__json_obj(url)

    if not json_obj:
      count=0
      while count < 3:    
        json_obj  =  get__json_obj(url)
        count += 1
        if json_obj:
          break

    if not json_obj:
      continue
    else:
      csv_list.append(get_elements_dict_from_url(count, url, json_obj))

In [17]:
def scrape_ted_urls(urls, file_name):
  """
    Realiza un proceso de web-scraping sobre todas las charlas TED

    Attributes
    ----------
    urls: list
      URL de charlas TED a scrappear
    file_name: list
      nombre de archivo donde se guardara toda la informacion de las charlas TED
  """
  csv_list_ = []
  
  with Manager() as manager:
      csv_list = manager.list()
      Processess = []
      
      urls_  = [urls[(i*(len(urls)//100)):((i+1)*(len(urls)//100))] for i in range(100)]
      
      leftovers = urls[(100*(len(urls)//100)):len(urls)]

      for i in range(len(leftovers)):
        urls_[i] += [leftovers[i]]
      
      for (id_,urls__) in enumerate(urls_):
          p = Process(target=download, args=(urls__,id_,csv_list))
          Processess.append(p)
          p.start()
          
      for t in Processess:
        t.join()
      
      csv_list_ = list(csv_list)

  dataframe_ted = pd.DataFrame(csv_list_)
  dataframe_ted = dataframe_ted.sort_values("view_count", ascending=False)
  dataframe_ted.to_csv(file_name, index=False, encoding='utf-8')

In [18]:
def get_page_text(page_number):
  """
    Obtiene la URL de una charla TED

    Attributes
    ----------
    page_number: int
      numero de pagina

    Returns
    -------
    str
      retorna la URL de una charla TED
  """
  res = requests.get("https://www.ted.com/talks?sort=popular&page=" + str(page_number), headers = {'User-agent': 'your bot 0.1'})
  soup = bs4.BeautifulSoup(res.text)
  element = soup.select("div.container.results div.col")
  return element

In [19]:
def retrieve_pages_url():
  """
    Obtiene las URL de todas las charlas TED en un determinado idioma

    Returns
    -------
    list
      retorna un listado de URL de cada charla TED en un determinado idioma
  """
  urls = []
  page_number=1

  while 1:
    page_list_urls = get_page_text(page_number)
    
    if len(page_list_urls) == 0:    
      break
    
    page_number += 1
    urls += get_urls(page_list_urls)

  file_ted = open('TED_Talk_URLs.txt', 'w')
  file_ted.write('\n'.join(urls))
  file_ted.close()

  return urls

#### 01.02 - Carga de Datos - Web Scrapping

In [20]:
# Activando Google Drive para guardar la información
drive.mount('/content/gdrive')

# Directorio donde se guardara/buscara el archivo que contiene la informacion de las charlas TED
os.chdir('/content/gdrive/My Drive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
# Si no existe el archivo de TED, hace toda la falopeada de web-scraping asquerosa copy-pasted (toma 6 minutos aprox)
if not os.path.isfile(ted_file):
  ted_urls = retrieve_pages_url()
  scrape_ted_urls(ted_urls, ted_file)

#### 01.03 - Exploración de los Datasets (DS)

In [22]:
tedx_df = pd.read_csv(ted_file)
tedx_df.head()

,language,talk__id,talk__name,talk__description,view_count,duration,transcript,video_type_name,event,speaker__id,speaker__name,speaker__description,speaker__who_he_is,speaker__why_listen,all_speakers_details,recording_date,published_timestamp,talks__tags,number_of__tags,native_language,url__webpage,talk__more_resources,number_of__talk__more_resources,talk__recommendations__blurb,talk__recommendations,number_of__talk__recommendations,related_talks,number_of__related_talks
0,en,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70672747,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,"['creativity', 'culture', 'dance', 'education'...",7,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,The education expert has a lot to say about th...,"[{'title': 'Reading list', 'description': '', ...",1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6
1,en,2405,This is what happens when you reply to spam email,"Suspicious emails: unclaimed insurance bonds, ...",63743542,588,"A few years ago, I got one of those spam email...",TED Stage Talk,TEDGlobal>Geneva,2664.0,James Veitch,"Comedian, writer","For James Veitch, spam emails proved the perfe...",James Veitch's correspondence with email spamm...,"[{'id': '2664', 'slug': 'james_veitch', 'is_pu...",2015-12-08,2020-10-23 13:51:31,"['comedy', 'curiosity', 'communication', 'humo...",5,en,https://www.ted.com/talks/james_veitch_this_is...,[],0,NaN,NaN,0,"[{'id': '2236', 'hero': 'https://pe.tedcdn.com...",4
2,en,1569,Your body language may shape who you are,(NOTE: Some of the findings presented in this ...,61595820,1262,So I want to start by offering you a free no-t...,TED Stage Talk,TEDGlobal 2012,1284.0,Amy Cuddy,Social psychologist,Amy Cuddy’s research on body language reveals ...,Amy Cuddy wasn’t supposed to become a successf...,"[{'id': '1284', 'slug': 'amy_cuddy', 'is_publi...",2012-06-26,2012-10-01 15:00:08,"['body language', 'brain', 'business', 'psycho...",7,en,https://www.ted.com/talks/amy_cuddy_your_body_...,"[{'status': 'approved', 'headline': 'Presence:...",4,NaN,NaN,0,"[{'id': '1042', 'hero': 'https://s3.amazonaws....",6
3,en,848,How great leaders inspire action,Simon Sinek has a simple but powerful model fo...,55008101,1084,How do you explain when things don't go as we ...,TEDx Talk,TEDxPuget Sound,703.0,Simon Sinek,Leadership expert,Simon Sinek explores how leaders can inspire c...,Fascinated by the leaders who make impact in t...,"[{'id': '703', 'slug': 'simon_sinek', 'is_publ...",2009-09-17,2010-05-04 09:31:00,"['TEDx', 'business', 'entrepreneur', 'leadersh...",5,en,https://www.ted.com/talks/simon_sinek_how_grea...,"[{'status': 'approved', 'headline': '*Start wi...",1,The leadership expert shares books and films t...,"[{'title': 'Reading List', 'description': '', ...",1,"[{'id': '1998', 'hero': 'https://pe.tedcdn.com...",6
4,en,1042,The power of vulnerability,Brené Brown studies human connection -- our ab...,53565659,1219,"So, I'll start with this: a couple years ago, ...",TEDx Talk,TEDxHouston,914.0,Brené Brown,"Vulnerability researcher, storyteller","Dr. Brené Brown studies courage, vulnerability...",Brené Brown is a research professor at the Uni...,"[{'id': '914', 'slug': 'brene_brown', 'is_publ...",2010-06-06,2010-12-23 14:45:00,"['TEDx', 'communication', 'culture', 'depressi...",10,en,https://www.ted.com/talks/brene_brown_the_powe...,"[{'status': 'approved', 'headline': '*Daring G...",2,Dive deeper into shame and vulnerability (in a...,"[{'title': 'Reading List', 'description': '', ...",1,"[{'id': '1391', 'hero': 'https://s3.amazonaws....",6


##### *Dataset (DS):*
 Se genera un DS de trabajo con mas de 5000 observaciones a partir de la aplicación de Web Scrapping en https://www.ted.com/talks?sort=popular&page=1, el ds de trabajo contempla tanto la metadata de las ted talks como las transcripciones.

Variables:

1. talk__id: N° de ID de la Ted Talk
2. talk__name: Nombre de la Ted Talk
3. talk__description: Descripción de la Ted Talk
4. view_count: Cantidad de Vistas (visitas)
5. duration: duración de la Ted Talk en minutos
6. transcript: transcripción de la Ted Talk
7. video_type_name: Tipo de Ted Talk
8. event: evento
9. speaker__id: id del speaker
10. speaker__name: nombre del speaker
11. speaker__description: ocupación del speaker
12. speaker__who_he_is: descripción del Speaker
13. speaker__why_listen: 
14. all_speakers_details: 
15. recording_date: 
16. published_timestamp: 
17. talks__tags: Tags (Variable Target)
18. number_of__tags: cantidad de Tags
19. language: Idioma
20. native_language
21. url__webpage
22. talk__more_resources
23. number_of__talk__more_resources
24. talk__recommendations__blurb
25. talk__recommendations
26. number_of__talk__recommendations
27. related_talks
28. number_of__related_talks

In [23]:
#Dimensiones
print(f"Dimensiones de DS : {tedx_df.shape}")

Dimensiones de DS : (5122, 28)


In [24]:
#Valores Repetidos
tedx_df.loc[: , ["talk__id"]].size - np.unique(tedx_df.loc[: , ["talk__id"]]).size #Se confirma que hay un (1) valor duplicado

1

In [25]:
#Verificamos Observación Repetida
def repetidos(x):
  y = pd.Series(np.sort(x.values.tolist(), axis = None)).value_counts()
  return y[y > 1]

repetidos(tedx_df.loc[: , ["talk__id"]])

36409    2
dtype: int64

In [26]:
tedx_df[tedx_df.talk__id == 36409] #Considerando que la transcripción es de tipo NaN, se eliminará el duplicado en el siguiente paso.

,language,talk__id,talk__name,talk__description,view_count,duration,transcript,video_type_name,event,speaker__id,speaker__name,speaker__description,speaker__who_he_is,speaker__why_listen,all_speakers_details,recording_date,published_timestamp,talks__tags,number_of__tags,native_language,url__webpage,talk__more_resources,number_of__talk__more_resources,talk__recommendations__blurb,talk__recommendations,number_of__talk__recommendations,related_talks,number_of__related_talks
3946,en,36409,An unexpected tool for understanding inequalit...,How do we make sense of a world that doesn't? ...,565270,679,NaN,TEDx Talk,TEDxLondon,5215.0,Eugenia Cheng,"Mathematician, pianist","Eugenia Cheng devotes her life to mathematics,...",Dr. Eugenia Cheng quit her tenured academic jo...,"[{'id': '5215', 'slug': 'eugenia_cheng', 'is_p...",2018-07-01,2019-03-08 15:47:37,"['math', 'communication', 'visualizations', 'T...",7,en,https://www.ted.com/talks/eugenia_cheng_an_une...,"[{'status': 'approved', 'headline': '*The Art ...",1,NaN,NaN,0,"[{'id': '23865', 'hero': 'https://s3.amazonaws...",6
3947,en,36409,An unexpected tool for understanding inequalit...,How do we make sense of a world that doesn't? ...,565270,679,"The world is awash with divisive arguments, co...",TEDx Talk,TEDxLondon,5215.0,Eugenia Cheng,"Mathematician, pianist","Eugenia Cheng devotes her life to mathematics,...",Dr. Eugenia Cheng quit her tenured academic jo...,"[{'id': '5215', 'slug': 'eugenia_cheng', 'is_p...",2018-07-01,2019-03-08 15:47:37,"['math', 'communication', 'visualizations', 'T...",7,en,https://www.ted.com/talks/eugenia_cheng_an_une...,"[{'status': 'approved', 'headline': '*The Art ...",1,NaN,NaN,0,"[{'id': '23865', 'hero': 'https://s3.amazonaws...",6


In [27]:
#Verificamos idioma:
def idiomas(x):
  y = pd.Series(np.sort(x.values.tolist(), axis = None)).value_counts()
  return y

idiomas(tedx_df.loc[: , ["language"]])

#Existen Ted Talks en distintos idiomas, seleccionamos solo las Ted Talks en Inglés

en    5102
es      19
fr       1
dtype: int64

In [28]:
tedx_df = tedx_df[tedx_df.language == 'en']

##### *Valores Ausentes / Missing Values:* 

Se observa para las siguientes variables una cantidad de NA's considerable (>= 0.5), por lo que se eliminarán del modelo.

Para la variable principal del modelo se observan 587 observaciones con datos faltantes, por ello se limpia el dataset.

In [29]:
#Todas las variables
tedx_df.isna().sum() / tedx_df.shape[0]

language                            0.000000
talk__id                            0.000000
talk__name                          0.000000
talk__description                   0.000000
view_count                          0.000000
duration                            0.000000
transcript                          0.116229
video_type_name                     0.000000
event                               0.000000
speaker__id                         0.002352
speaker__name                       0.000196
speaker__description                0.150137
speaker__who_he_is                  0.135045
speaker__why_listen                 0.229714
all_speakers_details                0.000000
recording_date                      0.000392
published_timestamp                 0.000000
talks__tags                         0.000000
number_of__tags                     0.000000
native_language                     0.000000
url__webpage                        0.000000
talk__more_resources                0.525872
number_of_

In [30]:
#Eliminamos NA's en transcription
tedx_df = tedx_df.dropna(subset = ['transcript'])
tedx_df.shape

(4509, 28)

In [31]:
y = tedx_df.isna().sum() / tedx_df.shape[0]
y[y >= 0.5].index

Index(['talk__recommendations__blurb', 'talk__recommendations'], dtype='object')

In [32]:
#Eliminamos Variables f > 0.5 NA's
tedx_df = tedx_df.drop(columns = y[y >= 0.5].index)

tedx_df.shape

(4509, 26)

#### 01.04 - Preprocesamiento de Dataset

Se toma como referencia el pipeline de: https://medium.com/analytics-vidhya/text-preprocessing-for-nlp-natural-language-processing-beginners-to-master-fd82dfecf95 y se complementa con parte del pipeline de: https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79 (no se corre el tratamiento de contracciones ni de números en forma de texto al no poder instalar pip en Kaggle)

##### *Limpieza URL's:* 
Deberia ser irrelevante debido a que son transcripciones directas de charlas, no deberian existir referencias a URL's dentro del texto. No obstante, se ejecuta este paso de forma preventiva.

In [33]:
def clean_url(text):
    return re.sub(r'http\S+','',text) #Definimos función (cortesia de bibliografía referencial)

tedx_df['CleanTranscript'] = tedx_df['transcript'].apply(clean_url)

##### *Limpieza de Tildes:* 
Deberia ser irrelevante debido a que son textos en Inglés, no obstante, se ejecuta este paso de forma preventiva.

In [34]:
def remove_tildes(text):
    text = unidecode.unidecode(text)
    return text

tedx_df['CleanTranscript'] = tedx_df['CleanTranscript'].apply(remove_tildes)

##### *Limpieza de Caracteres Irrelevantes:* 

Se eliminan números y signos de puntuación. Hay que considerar el impacto sobre it's, i'm, entre otros...

In [35]:
def clean_non_alphanumeric(text):
   return re.sub('[^a-zA-Z]',' ',text) #Definimos función (cortesia de bibliografía referencial)

tedx_df['CleanTranscript'] = tedx_df['CleanTranscript'].apply(clean_non_alphanumeric)

##### *Estandarización de tamaño de letras:* 

Se convierten todos los caracteres a minusculas

In [36]:
def clean_lowercase(text):
    return str(text).lower() #Definimos función (cortesia de bibliografía referencial)

tedx_df['CleanTranscript'] = tedx_df['CleanTranscript'].apply(clean_lowercase)

#### 01.05 - Tags

Referencia: https://www.kaggle.com/rounakbanik/ted-data-analysis

#### *Segmentación por TAGS:* 

In [37]:
tedx_df = tedx_df.assign(talks__tags=tedx_df['talks__tags'].str.split(',')).explode('talks__tags')
tedx_df['tag'] = remove_character(remove_character(remove_character(tedx_df['talks__tags'], "["), "'"), "]")
tedx_df.head()

,language,talk__id,talk__name,talk__description,view_count,duration,transcript,video_type_name,event,speaker__id,speaker__name,speaker__description,speaker__who_he_is,speaker__why_listen,all_speakers_details,recording_date,published_timestamp,talks__tags,number_of__tags,native_language,url__webpage,talk__more_resources,number_of__talk__more_resources,number_of__talk__recommendations,related_talks,number_of__related_talks,CleanTranscript,tag
0,en,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70672747,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,['creativity',7,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6,good morning how are you audience good it s...,creativity
0,en,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70672747,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,'culture',7,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6,good morning how are you audience good it s...,culture
0,en,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70672747,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,'dance',7,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6,good morning how are you audience good it s...,dance
0,en,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70672747,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,'education',7,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6,good morning how are you audience good it s...,education
0,en,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70672747,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,'parenting',7,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6,good morning how are you audience good it s...,parenting


In [38]:
tedx_df = tedx_df[tedx_df['transcript'] != np.nan]

In [39]:
tedx_df.shape

(33297, 28)

### 02.Modelado

#### 02.00 - Train / Test Split

In [50]:
X_train, X_test, y_train, y_test = train_test_split(tedx_df.transcript, tedx_df.tag, test_size=.30)
X_train = X_train.values.astype('U')
X_test = X_test.values.astype('U')
del tedx_df
gc.collect()

3427

In [51]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=pre_procesamiento_texto)

#### 02.01 - Stacking

In [52]:
estimators = [
    ('rf', RandomForestClassifier()),
    ('svr', LinearSVC()),
    ('nb', GaussianNB()),
    ('dt', DecisionTreeClassifier()),
    ('ab', AdaBoostClassifier())
]

model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [53]:
ml_pipeline = Pipeline(steps=[
                              ('preprocessor', tfidf_vectorizer),
                              ('classifier', model)
])

In [ ]:
ml_pipeline.fit(X_train, y_train)

In [ ]:
# TODO: Aplicar la función de similarity_btw_docs